[![mg/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

# La extensión *Flask-WTF*.

La extensión *Flask-WTF* permite deplegar y validar de formularios en una aplicación de *Flask* a partir de la biblioteca *WTForms*.

https://flask-wtf.readthedocs.io/en/1.0.x/

El paquete ```flask_wtf``` contiene los componentes necesarios para la gestión de formularios en *Flas*.

In [ ]:
!pip install flask-wtf

## La clase ```flask_wtf.FlaskForm```.

La clase ```flask_wtf.FlaskForm``` es la implementación en *Flask* de la clase ```wtforms_Form```, permitiendo incluir los objetos instanciados de dicha clase en una plantilla *HTML* de *Jinja*.

In [ ]:
from flask_wtf import FlaskForm

### El método ```flask_wtf.FlaskForm.hidden_tag()```.

Este método se utiliza para agregar una etiqueta oculta dentro de una forma. Por lo general se utiliza para evitar ataques de tipo *CSRF*.

### El método ```flask_wtf.FlaskForm.is_submitted()```.

Este método valida si la forma fue enviada.

### El método ```flask_wtf.FlaskForm.validate_on_submit()```.

Este método valida los datos de cada campo una vez que son enviados y regresa ```True``` en caso de que dicha validación haya sido exitosa.

## Protección contra ataques *CSRF*.

El paquete *wtforms* utiliza por defecto un mecanismo que impide que reciba peticiones que no pertenezcan al sitio. A esta técnica de penetración se le conoce como "*CSRF*", por las siglas en inglés de *Cross Site Request Forgery* o Falsificación de Peticiones Cruzadas entre Sitios.

Esta protección se realiza generando *tokens* cifrados que permiten verifican y validan que la autenticidad de los formularios que se reciben y se envían por parte de la aplicación. 

### El campo ```app.config['SECRET_KEY']```.

Los *tokens* cifrado son generados a partir de una cadena inicial, la cual es definida en el campo ```app.config['SECRET_KEY']```. 

La cadena de generación de *tokens* es un elemento crítico, ya que un atacante con acceso a dicha información podría vulnerar la autenticidad de las transascciones de la aplicación. por lo anterior, se recomienda que dicha cadena no sea visible en un entorno productivo y se extraiga de algún sistema de gestión de secretos o de bóveda (*vault*).

```
app.config['SECRET_KEY'] = <objeto tipo str>
```

## Aplicación ilustrativa.

### La plantilla ```plantilla_formularios.html```.

* La plantilla localizada en [```templates/plantilla_formularios.html```](templates/plantilla_formularios.html) tiene el siguiente contenido:

``` html
<!DOCTYPE html>
<html>
  <head>
      <title>Datos del alumno</title>
  </head>
  <body>
    <h1>Datos personales del alumno.</h1>
    <form method="POST">
      {{ form.hidden_tag() }}
      <p>{{ form.nombre.label }}</p>
      {{ form.nombre }}
      <p>{{ form.primer_apellido.label }}</p>
      {{ form.primer_apellido }}
      <p>{{ form.segundo_apellido.label }}</p>
      {{ form.segundo_apellido }}
      <p>{{ form.carrera.label }}</p>
      {{ form.carrera }}
      <p>{{ form.semestre.label }}</p>
      {{ form.semestre }}
      <p>{{ form.promedio.label }}</p>
      {{ form.promedio }}
      <p>{{ form.al_corriente.label }}</p>
      {{ form.al_corriente }}
      <p>
      {{ form.enviar }}
    </form>
  </body>
</html>
```

### Importación de componentes.

In [ ]:
from flask import Flask, render_template, abort
from flask_wtf import FlaskForm
from wtforms.validators import DataRequired, ValidationError
from wtforms.fields import SelectField, BooleanField, SubmitField, StringField

### Definición de carreras.

In [ ]:
carreras = ("Sistemas", 
            "Derecho", 
            "Actuaría", 
            "Arquitectura", 
            "Administración")

### Definición de campos.

In [ ]:
campos = ('nombre',
          'primer_apellido', 
          'segundo_apellido', 
          'carrera', 
          'semestre', 
          'promedio', 
          'al_corriente')

### La función ```valida_promedio()```.

In [ ]:
def valida_promedio(form, field):
    try:
        numero = float(field.data)
    except:
        raise ValidationError('Debe de ingresar un número')
    if numero < 0 or numero > 10:
        raise ValidationError('Debe de ingresar un número entre 0 y 10')        

### La clase ```DatosEstudiante```.

In [ ]:
class DatosEstudiante(FlaskForm):            
    nombre = StringField('Nombre', 
                         [DataRequired()], 
                         default = '')
    primer_apellido = StringField('Primer apellido', 
                                  [DataRequired()], 
                                  default = '')
    segundo_apellido = StringField('Segundo apellido', 
                                   default = '')
    carrera = SelectField('Carrera', 
                          [DataRequired()], 
                          choices = carreras)
    semestre = SelectField('Semestre', 
                           [DataRequired()], 
                           choices = [(str(x), str(x)) for x in range(1, 50)])
    promedio = StringField('Promedio', 
                           [DataRequired(), valida_promedio], 
                           default = '0')
    al_corriente = BooleanField('Al corriente de pagos')
    enviar = SubmitField('Enviar')

### Creación y configuración de la aplicación.

In [ ]:
app = Flask(__name__)
app.config['SECRET_KEY'] = "Pyth0n15t4"

### Cuerpo de la aplicación.

In [ ]:
@app.route('/', methods=['GET', 'POST'])
def index():
    forma = DatosEstudiante()
    if forma.validate_on_submit():
        for campo in campos:
            print(forma[campo].data) 
    else:
        print('Datos incorrectos.')
    return render_template('plantilla_formularios.html', form=forma)

**Advertencia:** Una vez ejecutada la siguiente celda, es necesario interrumpir el *kernel* de *Jupyter* para poder ejecutar el resto de las celdas de la *notebook*.

In [ ]:
#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run('0.0.0.0')

http://localhost:5000/

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>